<h1>HIGH LEVEL API - ESTIMATORS</h1>
<h2>Simple Linear Regression</h2>

In [1]:
import pandas as pd
base = pd.read_csv('house_prices.csv')
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
base.shape

(21613, 21)

In [3]:
x = base.iloc[:, 5:6].values # .values to make it numpy
y = base.iloc[:, 2:3].values

In [4]:
x

array([[1180],
       [2570],
       [ 770],
       ...,
       [1020],
       [1600],
       [1020]], dtype=int64)

In [5]:
y

array([[221900.],
       [538000.],
       [180000.],
       ...,
       [402101.],
       [400000.],
       [325000.]])

In [6]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

In [7]:
x

array([[-0.97983502],
       [ 0.53363434],
       [-1.42625404],
       ...,
       [-1.15404732],
       [-0.52252773],
       [-1.15404732]])

In [8]:
y

array([[-0.86671733],
       [-0.00568792],
       [-0.98084935],
       ...,
       [-0.37586519],
       [-0.38158814],
       [-0.58588173]])

In [9]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [10]:
columns = [tf.feature_column.numeric_column('x')]

In [11]:
columns

[NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [12]:
regressor = tf.estimator.LinearRegressor(columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\GABRIE~1.CAL\\AppData\\Local\\Temp\\tmprvh58o4d', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000218399478C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
# train/test rate - 70/30
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [14]:
print(f"x_train: {x_train.shape}, x_test: {x_test.shape}")

x_train: (15129, 1), x_test: (6484, 1)


In [15]:
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

y_train: (15129, 1), y_test: (6484, 1)


In [16]:
# set train func and inputs
train_func = tf.estimator.inputs.numpy_input_fn({'x': x_train}, 
                                                y_train, 
                                                batch_size=32, 
                                                num_epochs=None, 
                                                shuffle=True)

test_func = tf.estimator.inputs.numpy_input_fn({'x': x_test}, 
                                              y_test, 
                                              batch_size=32, 
                                              num_epochs=1000,
                                              shuffle=False)

In [17]:
type(train_func)

function

In [18]:
regressor.train(input_fn=train_func, steps=10000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\GABRIE~1.C

INFO:tensorflow:loss = 28.918453, step = 5101 (0.077 sec)
INFO:tensorflow:global_step/sec: 1170.33
INFO:tensorflow:loss = 8.835495, step = 5201 (0.086 sec)
INFO:tensorflow:global_step/sec: 1262.79
INFO:tensorflow:loss = 7.440279, step = 5301 (0.080 sec)
INFO:tensorflow:global_step/sec: 1392.07
INFO:tensorflow:loss = 17.945538, step = 5401 (0.071 sec)
INFO:tensorflow:global_step/sec: 1302.65
INFO:tensorflow:loss = 16.55635, step = 5501 (0.077 sec)
INFO:tensorflow:global_step/sec: 1285.46
INFO:tensorflow:loss = 8.730186, step = 5601 (0.080 sec)
INFO:tensorflow:global_step/sec: 1162.09
INFO:tensorflow:loss = 8.565279, step = 5701 (0.084 sec)
INFO:tensorflow:global_step/sec: 1341.06
INFO:tensorflow:loss = 11.285416, step = 5801 (0.075 sec)
INFO:tensorflow:global_step/sec: 1394.55
INFO:tensorflow:loss = 9.656384, step = 5901 (0.072 sec)
INFO:tensorflow:global_step/sec: 1404.96
INFO:tensorflow:loss = 9.643255, step = 6001 (0.071 sec)
INFO:tensorflow:global_step/sec: 1455.6
INFO:tensorflow:lo

In [19]:
# evaluate train metrics
train_metrics = regressor.evaluate(input_fn=train_func, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-08T17:06:04Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\GABRIE~1.CAL\AppData\Local\Temp\tmprvh58o4d\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Finished evaluation at 2021-12-08-17:06:09
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.49300063, global_step = 10000, label/mean = -0.008719033, loss = 15.77602, prediction/mean = -0.0012872613


In [20]:
# test database evaluation
test_metrics = regressor.evaluate(input_fn=test_func, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-08T17:06:09Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\GABRIE~1.CAL\AppData\Local\Temp\tmprvh58o4d\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Finished evaluation at 2021-12-08-17:06:15
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.5408701, global_step = 10000, label/mean = 0.02058878, loss = 17.307842, prediction/mean = 0.015146463
INFO

In [21]:
train_metrics

{'average_loss': 0.49300063,
 'label/mean': -0.008719033,
 'loss': 15.77602,
 'prediction/mean': -0.0012872613,
 'global_step': 10000}

In [22]:
test_metrics

{'average_loss': 0.5408701,
 'label/mean': 0.02058878,
 'loss': 17.307842,
 'prediction/mean': 0.015146463,
 'global_step': 10000}

<h1>Make new predictions using the  tested model:</h1>

In [23]:
import numpy as np
new_houses = np.array([[800], [900], [1000]])
new_houses

array([[ 800],
       [ 900],
       [1000]])

In [24]:
# scaler
new_houses = scaler_x.transform(new_houses)

In [25]:
new_houses

array([[-1.39358923],
       [-1.28470655],
       [-1.17582386]])

In [26]:
predict_func = tf.estimator.inputs.numpy_input_fn({'x': new_houses}, shuffle=False)

In [27]:
predicts = regressor.predict(input_fn=predict_func)

In [28]:
predicts

<generator object Estimator.predict at 0x000002183B34B848>

In [29]:
list(predicts)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\GABRIE~1.CAL\AppData\Local\Temp\tmprvh58o4d\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([-0.95859766], dtype=float32)},
 {'predictions': array([-0.8834211], dtype=float32)},
 {'predictions': array([-0.80824447], dtype=float32)}]

In [35]:
for p in regressor.predict(input_fn=predict_func):
    # print(p['predictions'])
    print(scaler_y.inverse_transform([p['predictions']]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\GABRIE~1.CAL\AppData\Local\Temp\tmprvh58o4d\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[188169.01193088]]
[[215767.72471753]]
[[243366.48126814]]
